# FlightAI Assistant – Amadeus Integration

Business application of the Airline AI Assistant (week2/day4): real-time **flight search** and **booking info** via the Amadeus API, with credentials from a `.env` file.

In [10]:
import os
import json
import time
import requests
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [11]:
# Load environment variables from .env (here or project root)
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    env_file = p / ".env"
    if env_file.exists():
        load_dotenv(env_file, override=True)
        break
else:
    load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key loaded (starts with {openai_api_key[:8]}...)")
else:
    print("OPENAI_API_KEY not set in .env")

MODEL = "gpt-4.1-mini"
openai_client = OpenAI()

# Amadeus: use requests instead of SDK (avoids NetworkError with SDK)
AMADEUS_BASE = "https://test.api.amadeus.com"
_amadeus_token = None
_amadeus_token_expiry = 0

def get_amadeus_token():
    """Get Amadeus OAuth2 access token (cached until near expiry)."""
    global _amadeus_token, _amadeus_token_expiry
    if _amadeus_token and time.time() < _amadeus_token_expiry - 60:
        return _amadeus_token
    cid = os.getenv('AMADEUS_CLIENT_ID')
    sec = os.getenv('AMADEUS_CLIENT_SECRET')
    if not cid or not sec:
        return None
    r = requests.post(
        f"{AMADEUS_BASE}/v1/security/oauth2/token",
        headers={"Content-Type": "application/x-www-form-urlencoded"},
        data={"grant_type": "client_credentials", "client_id": cid, "client_secret": sec},
        timeout=10
    )
    if r.status_code != 200:
        return None
    data = r.json()
    _amadeus_token = data.get("access_token")
    _amadeus_token_expiry = time.time() + data.get("expires_in", 1799)
    return _amadeus_token

amadeus_configured = bool(os.getenv('AMADEUS_CLIENT_ID')) and bool(os.getenv('AMADEUS_CLIENT_SECRET'))
if amadeus_configured:
    tok = get_amadeus_token()
    print("Amadeus configured (requests-based). Token:", "OK" if tok else "FAILED")
else:
    print("AMADEUS_CLIENT_ID or AMADEUS_CLIENT_SECRET missing in .env – flight tools will return errors.")

OpenAI API Key loaded (starts with sk-proj-...)
Amadeus configured (requests-based). Token: OK


## Amadeus tools (real-time flights & bookings)

- **search_flights**: Flight offers between two airports on a date.
- **flight_inspiration**: Cheapest destinations from an origin (IATA code).
- **flight_cheapest_dates**: Cheapest dates for a given route.
- **get_flight_order**: Retrieve a booking by order ID.

In [12]:
def search_flights(origin_code, destination_code, departure_date, adults=1):
    """Search real-time flight offers via Amadeus. Use IATA airport codes (e.g. LHR, JFK)."""
    print(f"[Tool called] search_flights(origin={origin_code}, destination={destination_code}, date={departure_date}, adults={adults})", flush=True)
    token = get_amadeus_token()
    if not token:
        return "Amadeus is not configured or token failed. Add AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET to your .env file."
    try:
        r = requests.get(
            f"{AMADEUS_BASE}/v2/shopping/flight-offers",
            params={
                "originLocationCode": origin_code.upper(),
                "destinationLocationCode": destination_code.upper(),
                "departureDate": departure_date,
                "adults": int(adults),
            },
            headers={"Authorization": f"Bearer {token}"},
            timeout=15,
        )
        if r.status_code != 200:
            return f"Amadeus API error: {r.status_code} - {r.text[:200]}"
        data = r.json().get("data", [])
        if not data:
            return f"No flight offers found for {origin_code} to {destination_code} on {departure_date}."
        lines = []
        for i, offer in enumerate(data[:5], 1):
            price = offer.get("price", {}).get("total", "N/A")
            currency = offer.get("price", {}).get("currency", "")
            segments = offer.get("itineraries", [{}])[0].get("segments", [])
            dep = segments[0].get("departure", {}).get("at", "")[:16] if segments else ""
            arr = segments[-1].get("arrival", {}).get("at", "")[:16] if segments else ""
            lines.append(f"{i}. {price} {currency} | Dep: {dep} | Arr: {arr}")
        return "Flight offers: " + "; ".join(lines)
    except requests.RequestException as e:
        return f"Amadeus request error: {e}"

def flight_inspiration(origin_code):
    """Get cheapest destinations from an origin (IATA code, e.g. LON or LHR)."""
    token = get_amadeus_token()
    if not token:
        return "Amadeus is not configured or token failed. Add AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET to your .env file."
    try:
        r = requests.get(
            f"{AMADEUS_BASE}/v1/shopping/flight-destinations",
            params={"origin": origin_code.upper()},
            headers={"Authorization": f"Bearer {token}"},
            timeout=15,
        )
        if r.status_code != 200:
            return f"Amadeus API error: {r.status_code} - {r.text[:200]}"
        data = r.json().get("data", [])
        if not data:
            return f"No inspiration results for origin {origin_code}."
        lines = [f"{d.get('destination', '')}: {d.get('price', 'N/A')} {d.get('currency', '')}" for d in data[:10]]
        return "Cheapest destinations from " + origin_code + ": " + "; ".join(lines)
    except requests.RequestException as e:
        return f"Amadeus request error: {e}"

def flight_cheapest_dates(origin_code, destination_code):
    """Get cheapest travel dates for a route (IATA codes)."""
    token = get_amadeus_token()
    if not token:
        return "Amadeus is not configured or token failed. Add AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET to your .env file."
    try:
        r = requests.get(
            f"{AMADEUS_BASE}/v1/shopping/flight-dates",
            params={"origin": origin_code.upper(), "destination": destination_code.upper()},
            headers={"Authorization": f"Bearer {token}"},
            timeout=15,
        )
        if r.status_code != 200:
            return f"Amadeus API error: {r.status_code} - {r.text[:200]}"
        data = r.json().get("data", [])
        if not data:
            return f"No date results for {origin_code} to {destination_code}."
        lines = [f"{d.get('departureDate', '')}: {d.get('price', 'N/A')} {d.get('currency', '')}" for d in data[:7]]
        return "Cheapest dates: " + "; ".join(lines)
    except requests.RequestException as e:
        return f"Amadeus request error: {e}"

def get_flight_order(order_id):
    """Retrieve a flight booking by its order ID (from Amadeus Flight Create Orders)."""
    token = get_amadeus_token()
    if not token:
        return "Amadeus is not configured or token failed. Add AMADEUS_CLIENT_ID and AMADEUS_CLIENT_SECRET to your .env file."
    try:
        r = requests.get(
            f"{AMADEUS_BASE}/v1/booking/flight-orders/{order_id}",
            headers={"Authorization": f"Bearer {token}"},
            timeout=15,
        )
        if r.status_code != 200:
            return f"Amadeus API error: {r.status_code} - {r.text[:200]}"
        data = r.json().get("data", r.json())
        if not data:
            return f"No order found for ID: {order_id}"
        s = json.dumps(data, indent=2)
        return s[:1500] + ("..." if len(s) > 1500 else "")
    except requests.RequestException as e:
        return f"Amadeus request error: {e}"

In [13]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_flights",
            "description": "Search for real-time flight offers from an origin airport to a destination airport on a specific date. Use IATA codes (e.g. LHR, JFK, MAD).",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_code": { "type": "string", "description": "Origin airport/city IATA code" },
                    "destination_code": { "type": "string", "description": "Destination airport/city IATA code" },
                    "departure_date": { "type": "string", "description": "Departure date YYYY-MM-DD" },
                    "adults": { "type": "integer", "description": "Number of adult passengers", "default": 1 }
                },
                "required": ["origin_code", "destination_code", "departure_date"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "flight_inspiration",
            "description": "Get cheapest destinations from an origin (IATA code, e.g. LON, MAD). Use when the user asks for ideas or cheap places to fly from a city.",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_code": { "type": "string", "description": "Origin city/airport IATA code" }
                },
                "required": ["origin_code"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "flight_cheapest_dates",
            "description": "Get cheapest travel dates for a specific route (origin and destination IATA codes).",
            "parameters": {
                "type": "object",
                "properties": {
                    "origin_code": { "type": "string", "description": "Origin IATA code" },
                    "destination_code": { "type": "string", "description": "Destination IATA code" }
                },
                "required": ["origin_code", "destination_code"],
                "additionalProperties": False
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_flight_order",
            "description": "Retrieve details of a flight booking by its Amadeus order ID. Use when the user asks about a specific booking or order status.",
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": { "type": "string", "description": "The flight order/booking ID" }
                },
                "required": ["order_id"],
                "additionalProperties": False
            }
        }
    }
]

In [ ]:
def handle_tool_calls(message):
    responses = []
    for tool_call in message.tool_calls:
        name = tool_call.function.name
        try:
            arguments = json.loads(tool_call.function.arguments)
        except json.JSONDecodeError:
            arguments = {}
        if name == "search_flights":
            content = search_flights(
                arguments.get("origin_code", ""),
                arguments.get("destination_code", ""),
                arguments.get("departure_date", ""),
                arguments.get("adults", 1)
            )
        elif name == "flight_inspiration":
            content = flight_inspiration(arguments.get("origin_code", ""))
        elif name == "flight_cheapest_dates":
            content = flight_cheapest_dates(
                arguments.get("origin_code", ""),
                arguments.get("destination_code", "")
            )
        elif name == "get_flight_order":
            content = get_flight_order(arguments.get("order_id", ""))
        else:
            content = f"Unknown tool: {name}"
        responses.append({
            "role": "tool",
            "content": content,
            "tool_call_id": tool_call.id
        })
    return responses

In [14]:
system_message = """
You are a helpful assistant for FlightAI airline. You have access to real-time flight data via Amadeus.
Use the tools to search flights, get inspiration (cheapest destinations), cheapest dates for a route, or look up a booking by order ID.
Use IATA airport/city codes (e.g. LHR, JFK, MAD, LON). Be concise and accurate. If you don't know, say so.
"""

def chat(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [
        {"role": "system", "content": system_message},
        *history,
        {"role": "user", "content": message}
    ]
    response = openai_client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools
    )
    while response.choices[0].finish_reason == "tool_calls":
        msg = response.choices[0].message
        tool_responses = handle_tool_calls(msg)
        messages.append(msg)
        messages.extend(tool_responses)
        response = openai_client.chat.completions.create(
            model=MODEL,
            messages=messages,
            tools=tools
        )
    return response.choices[0].message.content

In [15]:
gr.ChatInterface(fn=chat).launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


[Tool called] search_flights(origin=ABV, destination=LON, date=2026-03-13, adults=1)
